In [1]:
import os
import time
import socket
import struct
import pickle

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models

import torch.nn as nn
import torch.nn.functional as F


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

total_batch = len(trainloader)
print(total_batch)
print("Trainset size", trainloader.dataset.data.shape)

100%|██████████| 170498071/170498071 [00:20<00:00, 8255874.17it/s] 


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
12500
Trainset size (50000, 32, 32, 3)


In [3]:
class Fire(nn.Module):
    def __init__(self, in_channels, squeeze_channels, expand1x1_channels, expand3x3_channels):
        super(Fire, self).__init__()
        self.squeeze = nn.Conv2d(in_channels, squeeze_channels, kernel_size=1)
        self.expand1x1 = nn.Conv2d(squeeze_channels, expand1x1_channels, kernel_size=1)
        self.expand3x3 = nn.Conv2d(squeeze_channels, expand3x3_channels, kernel_size=3, padding=1)

    def forward(self, x):
        x = F.relu(self.squeeze(x))
        return torch.cat([
            F.relu(self.expand1x1(x)),
            F.relu(self.expand3x3(x))
        ], 1)

In [4]:
class ClientSqueezeNet1_1(nn.Module):
    def __init__(self):
        super(ClientSqueezeNet1_1, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=2)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)
        self.fire2 = Fire(64, 16, 64, 64)
        self.fire3 = Fire(128, 16, 64, 64)
        self.fire4 = Fire(128, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)
        self.fire5 = Fire(256, 32, 128, 128)
        self.fire6 = Fire(256, 48, 192, 192)
        self.fire7 = Fire(384, 48, 192, 192)
        self.fire8 = Fire(384, 64, 256, 256)
        self.maxpool8 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)
        self.fire9 = Fire(512, 64, 256, 256)
        self.conv10 = nn.Conv2d(512, 1000, kernel_size=1)
        self.avgpool10 = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.maxpool1(x)
        x = self.fire2(x)
        x = self.fire3(x)
        x = self.fire4(x)
        # x = self.maxpool4(x)
        # x = self.fire5(x)
        # x = self.fire6(x)
        # x = self.fire7(x)
        # x = self.fire8(x)
        # x = self.maxpool8(x)
        # x = self.fire9(x)
        # x = self.conv10(x)
        # x = self.avgpool10(x)
        # x = x.view(x.size(0), -1)  # Flatten for fully connected layer
        return x

# Create the model
client_model = ClientSqueezeNet1_1()
client_model = client_model.to(device)
print(client_model)

ClientSqueezeNet1_1(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (fire2): Fire(
    (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
    (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
    (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (fire3): Fire(
    (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
    (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
    (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (fire4): Fire(
    (squeeze): Conv2d(128, 32, kernel_size=(1, 1), stride=(1, 1))
    (expand1x1): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1))
    (expand3x3): Conv2d(32, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (maxpool4): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (fire5): Fire(
    (squeeze):

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(client_model.parameters(), lr=0.001, momentum=0.9)

In [6]:
def send_msg(sock, msg):
    # prefix each message with a 4-byte length in network byte order
    msg = pickle.dumps(msg)
    msg = struct.pack('>I', len(msg)) + msg
    sock.sendall(msg)

def recv_msg(sock):
    # read message length and unpack it into an integer
    raw_msglen = recvall(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # read the message data
    msg =  recvall(sock, msglen)
    msg = pickle.loads(msg)
    return msg

def recvall(sock, n):
    # helper function to receive n bytes or return None if EOF is hit
    data = b''
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data += packet
    return data

In [7]:
host = 'localhost'
port = 12305
s = socket.socket()
s.connect((host, port))
epoch = recv_msg(s)   # get epoch
msg = total_batch
send_msg(s, msg)   # send total_batch of train dataset

In [8]:
from tqdm import tqdm

for e in range(epoch):
    client_weights = recv_msg(s)
    client_model.load_state_dict(client_weights, strict=False)
    client_model.eval()
    for i, data in enumerate(tqdm(trainloader, ncols=100, desc='Epoch '+str(e+1))):
        x, label = data
        x = x.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        output = client_model(x)
        client_output = output.clone().detach().requires_grad_(True)
        msg = {
            'client_output': client_output,
            'label': label
        }
        send_msg(s, msg)
        client_grad = recv_msg(s)
        output.backward(client_grad)
        optimizer.step()
    send_msg(s, client_model.state_dict())

Epoch 1: 100%|████████████████████████████████████████████████| 12500/12500 [03:04<00:00, 67.66it/s]


In [9]:
torch.save(client_model.state_dict(), 'client_model_state_dict.pth')

# Replace 'path_to_file' with your file path
file_path = 'client_model_state_dict.pth'

# Get the size of the file in bytes
file_size = os.path.getsize(file_path)

print(f"The size of the file is {file_size / 1024} kb.")


The size of the file is 4844.431640625 kb.


In [10]:
from thop import profile
x =  torch.randn(50000, 3, 32, 32)
x = x.to(device)
macs, params = profile(client_model, inputs=(x,))
print(macs, params)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
187529600000.0 70976.0
